# Co-expression simulation

Building from the example here https://docs.pymc.io/notebooks/LKJ.html and here https://github.com/aloctavodia/Statistical-Rethinking-with-Python-and-PyMC3/blob/master/Chp_13.ipynb. And http://www.statsmodels.org/0.6.1/_modules/statsmodels/stats/moment_helpers.html


This notebook was written with a ton of help from Professor Eaton, thank you!

Update 5/11. Ugh. Toyplot seems to be introducing some kind of bug into my code that kills everything that runs after it. Incredibly encouraging!

In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import toyplot
from statsmodels.stats.moment_helpers import corr2cov, cov2corr

C:\Users\Alexander\Miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Define the shape of your simulated data

Each row represent a variable (gene) and columns are the observations.

In [2]:
# dimensions of the data
ncells = 500
ngenes = 10

### Covariance
We want to simulate data that has a correlation structure to it, but a correlation matrix itself does not provide sufficient information to do this, since data can be distributed in a number of ways and still have the same correlation for two values. Instead we need to generate a variance covariance matrix, which combines a correlation matrix and the standard deviation of those correlations. Formula for conversion of correlation matrix to VCV was found here [http://www.statsmodels.org/0.6.1/_modules/statsmodels/stats/moment_helpers.html](http://www.statsmodels.org/0.6.1/_modules/statsmodels/stats/moment_helpers.html). 

To simulate data to test the model I generate a correlation matrix, and sample means and standard deviations for each gene. From the correlation matrix and stdevs I generate a variance-covariance matrix. With sample means and a variance-covariance matrix we can now generate data that has a covariance structure by using the `np.random.multivariate_normal()` sampler. 

In [3]:
# fix random seed
np.random.seed(333)

# mean expression of each gene (normalized read counts) from uniform distribution  
mu = np.random.uniform(5, 10, ngenes)

# standard deviation of expression
estds = np.random.uniform(0, 2, ngenes)

# correlation matrix: use broadcasting to fill a symmetric matrix 
# with random uniform values in (-.99, .99) and set the diag to 1.
cor = np.random.uniform(-0.99, 0.99, ngenes)
cor = cor[:, None] * cor
np.fill_diagonal(cor, 1)
pd.DataFrame(cor).round(2)

# variance-covariance matrix from stds
print('variance-covariance matrix')
vcv = np.array(np.diag(estds) * np.matrix(cor) * np.diag(estds))
pd.DataFrame(vcv).round(2)

variance-covariance matrix


,0,1,2,3,4,5,6,7,8,9
0,0.01,-0.00,-0.02,-0.01,-0.01,-0.02,-0.07,0.04,-0.05,0.07
1,-0.00,0.01,0.00,0.00,0.00,0.00,0.01,-0.00,0.00,-0.01
2,-0.02,0.00,0.23,0.11,0.11,0.15,0.56,-0.32,0.40,-0.53
3,-0.01,0.00,0.11,0.17,0.10,0.13,0.51,-0.29,0.36,-0.48
4,-0.01,0.00,0.11,0.10,0.12,0.13,0.51,-0.29,0.36,-0.48
5,-0.02,0.00,0.15,0.13,0.13,0.69,0.67,-0.39,0.48,-0.64
6,-0.07,0.01,0.56,0.51,0.51,0.67,3.55,-1.46,1.81,-2.42
7,0.04,-0.00,-0.32,-0.29,-0.29,-0.39,-1.46,0.87,-1.04,1.39
8,-0.05,0.00,0.40,0.36,0.36,0.48,1.81,-1.04,3.69,-1.72
9,0.07,-0.01,-0.53,-0.48,-0.48,-0.64,-2.42,1.39,-1.72,4.00


In [4]:
# plot the correlation and covariance matrices
canvas = toyplot.Canvas(width=900, height=300)

canvas.matrix(
    cor,
    tlabel="correlation", 
    bounds=(25, 275, 25, 275));

canvas.matrix(
    vcv,
     tlabel="covariance", 
     bounds=(300, 550, 25, 275));

axes = canvas.cartesian(
    bounds=(600, 800, 55, 235), 
    ylabel="genes",
    xlabel="standard deviation")
axes.bars(estds[::-1], along='y', color=toyplot.color.brewer.map("BlueRed", reverse=True));

0 1 2 3 4 5 6 7 8 9 0 1.0 -0.013993167944131368 -0.3310689538825564 -0.350106124745729 -0.40612625547036024 -0.22702073555874028 -0.38017310660604403 0.4396942370678067 -0.2649994895609476 0.33933127673260605 1 -0.013993167944131368 1.0 0.023071146580146716 0.024397786708448686 0.028301652148653725 0.01582035586661026 0.02649305942305066 -0.03064089844388044 0.0184669749175379 -0.02364692168479036 2 -0.3310689538825564 0.023071146580146716 1.0 0.5772352447183646 0.6695980786775605 0.37429899274560235 0.6268079896815854 -0.7249430746732066 0.43691622166864524 -0.5594702826395798 3 -0.350106124745729 0.024397786708448686 0.5772352447183646 1.0 0.7081013961404199 0.3958219830328136 0.6628507857759696 -0.7666288474307551 0.4620398361521356 -0.5916410169792768 4 -0.40612625547036024 0.028301652148653725 0.6695980786775605 0.7081013961404199 1.0 0.4591570625013183 0.7689128767978358 -0.8892963622636791 0.5359703680445103 -0.686308904144359 5 -0.22702073555874028 0.01582035586661026 0.37429899274560235 0.3958219830328136 0.4591570625013183 1.0 0.4298150255492933 -0.4971082553059576 0.29960236638784266 -0.38364018612615 6 -0.38017310660604403 0.02649305942305066 0.6268079896815854 0.6628507857759696 0.7689128767978358 0.4298150255492933 1.0 -0.8324666435162573 0.5017196429033559 -0.6424509242274111 7 0.4396942370678067 -0.03064089844388044 -0.7249430746732066 -0.7666288474307551 -0.8892963622636791 -0.4971082553059576 -0.8324666435162573 1.0 -0.5802704919812346 0.7430351176165703 8 -0.2649994895609476 0.0184669749175379 0.43691622166864524 0.4620398361521356 0.5359703680445103 0.29960236638784266 0.5017196429033559 -0.5802704919812346 1.0 -0.447820122017848 9 0.33933127673260605 -0.02364692168479036 -0.5594702826395798 -0.5916410169792768 -0.686308904144359 -0.38364018612615 -0.6424509242274111 0.7430351176165703 -0.447820122017848 1.0 correlation 0 1 2 3 4 5 6 7 8 9 0 0.010655045246068275 -0.00017472098336007616 -0.01631742020166027 -0.01472301193195488 -0.014633486456671702 -0.01946648266279971 -0.0739127202120393 0.0423834695733667 -0.05251871063574179 0.07004072746225694 1 -0.00017472098336007613 0.014631967888282969 0.0013325267618861556 0.0012023228655288838 0.0011950119615862815 0.0015896881239434025 0.006035922131632357 -0.003461154200242529 0.004288826699134732 -0.005719724234100173 2 -0.016317420201660267 0.0013325267618861556 0.22798736736976435 0.11228649837934682 0.11160372353804704 0.14846304438726451 0.5637026293709111 -0.32324169874723696 0.4005391114223323 -0.5341724958878534 3 -0.014723011931954882 0.0012023228655288838 0.1122864983793468 0.16597344678227968 0.10069869703631666 0.13395641878154482 0.5086221005363223 -0.291657095836553 0.3614016213228106 -0.48197741637362346 4 -0.014633486456671702 0.0011950119615862815 0.11160372353804704 0.10069869703631666 0.12184793904348083 0.13314187674937938 0.5055293478101485 -0.2898836312597969 0.35920406473136246 -0.47904668063313566 5 -0.019466482662799715 0.0015896881239434025 0.1484630443872645 0.13395641878154482 0.13314187674937938 0.6900628117213846 0.6724903401401 -0.3856233918592583 0.4778382594745395 -0.6372612521853991 6 -0.0739127202120393 0.006035922131632357 0.5637026293709111 0.5086221005363223 0.5055293478101485 0.6724903401401 3.547482732758016 -1.4641820180580927 1.8143146962364447 -2.4196313967270626 7 0.0423834695733667 -0.0034611542002425293 -0.32324169874723696 -0.291657095836553 -0.2898836312597969 -0.3856233918592583 -1.4641820180580924 0.8720395183665706 -1.0403750735171025 1.3874793592732628 8 -0.05251871063574179 0.004288826699134732 0.4005391114223323 0.3614016213228106 0.35920406473136246 0.4778382594745395 1.8143146962364447 -1.0403750735171027 3.686232617765156 -1.7192699822887254 9 0.07004072746225694 -0.005719724234100173 -0.5341724958878534 -0.4819774163736235 -0.47904668063313566 -0.637261252185399 -2.4196313967270626 1.3874793592732628 -1.7192699822887254 3.998510269166099 covariance 0.0 0.5 1.0 1.5 2.0 standard deviation 0 5 10 genes

### Simulate data from this VCV

In [5]:
# set seed
np.random.seed(12345)

# sample data from this distribution
data = np.random.multivariate_normal(mu, vcv, size=ncells, check_valid='ignore')

#### Visualize the data matrix

In [6]:
# transposed to look like Alex's data file
canvas, table = toyplot.matrix(
    data[::10].T,
    lshow=False,
    tshow=False,
    width=400, 
    height=400, 
    tlabel="cells",
    llabel="genes", 
    colorshow=True,
);

# png format is less memory intensive
canvas.autorender(autoformat='png')

Exception: A ghostscript executable is required.

#### Visualize correlation of gene expression for two genes across cells
Hover over plots to see pop-ups. 

In [7]:
# what does the correlation look for gene expression of cells 0 and 1
canvas = toyplot.Canvas(width=600, height=300)

# which genes to plot
x, y = 6, 7

# create the axes
ax1 = canvas.cartesian(
    bounds=(50, 250, 50, 250), 
    xlabel="cell {} expression".format(x),
    ylabel="cell {} expression".format(y),
    label="expression x gene for cells {} v {}".format(x, y)
    )
ax2 = canvas.cartesian(
    bounds=(350, 550, 50, 250), 
    xlabel="gene {} expression".format(x),
    ylabel="gene {} expression".format(y),
    label="expression x cell for genes {} v {}".format(x, y),
    )

# add scatterplots to axes
ax1.scatterplot(
    data[x, :], data[y, :],
    size=10,
    title=["gene-{}".format(i) for i in range(ngenes)],
    color=toyplot.color.Palette()[1],
    );
ax2.scatterplot(
    data[:, x], data[:, y],
    size=10,
    title=["cell-{}".format(i) for i in range(ncells)],
    );

# png format is less memory intensive
canvas.autorender(autoformat='png')

Exception: A ghostscript executable is required.

Exception: A ghostscript executable is required.

### Estimate covariance matrix using Pymc3
https://docs.pymc.io/notebooks/LKJ.html. This is done using a prior on the covariance matrix that is defined by sampling standard deviations among samples and converting this to a lower triangle representation of the VCV. This convoluted method makes it quite fast. 

In [8]:
x = data
xsize = x.shape[1]
xmeans = x.mean(axis=0)

Exception: A ghostscript executable is required.

Exception: A ghostscript executable is required.

In [9]:
with pm.Model() as model:
    
    # distribution to draw stds from (not a rv for now) but is a wide prior
    stds = pm.HalfCauchy.dist(2)
    
    # sample correlations by drawing halfCauchy std values
    # eta=1 is uniform corrs, eta>1 puts more weight on corr=0.
    packed = pm.LKJCholeskyCov('packed_chol', n=xsize, eta=1., sd_dist=stds)
    
    # expand correlation matrix and define VCV as dot product
    chol = pm.expand_packed_triangular(xsize, packed)
    cov = pm.Deterministic('cov', chol.dot(chol.T))
    sigma = pm.Deterministic('sigma', tt.sqrt(tt.diag(cov)))

    # draw the means from a normal with starting values from observed means
    # set this mean and std of this dist according to your data, or it could
    # be made into a random variable to be fit.
    means = pm.Normal('means', 5, 5, shape=xsize, testval=xmeans)
    obs = pm.MvNormal('obs', means, chol=chol, observed=x)

Exception: A ghostscript executable is required.

Exception: A ghostscript executable is required.

In [10]:
with model:
    trace = pm.sample(tune=1000, draws=5000, njobs=3, cores=3)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


KeyboardInterrupt: 

Exception: A ghostscript executable is required.

Exception: A ghostscript executable is required.

### Examine traceplot
This looks like a really good match to the simulated data. Means were drawn uniformly from 20-100 and stds were drawn uniformly from 0-10. Correlations were drawn uniformly between -0.9 and 0.9, though these seem slighly underestimated. I think it may help to standardize the data before analysis, for example scaling expression to be in the range -1 to 1. 

In [ ]:
# plot the trace 
pm.traceplot(trace[1000::5], varnames=["means", "sigma", "cov"]);

In [ ]:
### Look at posterior parameter estimates
#pm.summary(trace[1000::2], varnames=['cov'])

### Show posterior means

In [ ]:
evcv = np.mean(trace[1000::5]["cov"], axis=0)
mvcv = np.median(trace[1000::5]["cov"], axis=0)
pd.DataFrame(mvcv).round(2)

In [ ]:
# plot matrices on the same scale and colorbar
cm = toyplot.color.brewer.map("BlueRed", domain_min=vcv.min(), domain_max=vcv.max())
canvas = toyplot.Canvas(width=900, height=300)
canvas.matrix((vcv, cm), tlabel="true vcv", bounds=(25, 275, 25, 275));
canvas.matrix((mvcv, cm), tlabel="estimated vcv", bounds=(300, 550, 25, 275));
canvas.matrix((vcv-mvcv, cm), step=5, tlabel="error", bounds=(575, 825, 25, 275));

### Did the chains converge?

In [ ]:
max(np.max(gr_stats) for gr_stats in pm.gelman_rubin(trace).values())

### Posterior has estimates of mean expression and correlation

In [ ]:
# true stdevs
estds

In [ ]:
# estimated stdevs
trace["sigma"].mean(axis=0)

In [ ]:
toyplot.scatterplot(
    estds, trace["sigma"].mean(axis=0), 
    width=300, 
    height=300, 
    size=10,
    xlabel="true stdev of genen expression",
    ylabel="estimated stdevs of gene expression",
);

### Convert vcv to corr

In [ ]:
C = cov2corr(mvcv)
pd.DataFrame(C).round(2)

In [ ]:
# true correlation
pd.DataFrame(cor).round(2)

In [ ]:
canvas, table = toyplot.matrix(C, tlabel="co-expression", width=400, height=400);

In [ ]:
# export final image
import toyplot.svg
toyplot.svg.render(canvas, "cormatrix.svg")